# Wyszukiwarka semantyczna wykorzystująca SVD

### Autor: Jakub Psarski

## 1. Wprowadzenie

Projekt zawiera implementację wyszukiwarki tekstowej z wykorzystaniem dekompozycji według wartości osobliwych (Singular Value Decomposition/SVD). Celem projektu było stworzenie efektywnego systemu wyszukiwania dokumentów, z możliwością porównania tradycyjnej metody opartej na podobieństwie cosinusowym wektorów TF-IDF oraz metody wykorzystującej redukcję wymiarowości poprzez SVD (znaną również jako Latent Semantic Indexing/LSI).

## 2. Struktura projektu

Projekt został podzielony na następujące moduły:

1. **Moduł przetwarzania tekstu** (`src/preprocessing.py`):
   - Budowanie słownika terminów
   - Tworzenie macierzy term-document
   - Implementacja transformacji IDF

2. **Moduł SVD** (`src/svd.py`):
   - Implementacja dekompozycji według wartości osobliwych
   - Redukcja wymiarowości macierzy

3. **Moduł wyszukiwania** (`src/search.py`):
   - Konwersja zapytań na wektory
   - Implementacja wyszukiwania bez SVD
   - Implementacja wyszukiwania z wykorzystaniem SVD

4. **Moduł konfiguracyjny** (`src/setup.py`):
   - Konfiguracja i inicjalizacja komponentów wyszukiwarki
   - Zarządzanie zapisem i odczytem obliczonych danych

5. **Interfejs konsolowy** (`main.py`):
   - Obsługa poleceń użytkownika
   - Wyświetlanie wyników wyszukiwania

6. **Interfejs webowy** (`app.py` oraz katalog `client`):
   - REST API w Flasku
   - Interfejs użytkownika w HTML/CSS/JavaScript

7. **Web crawler** (`src/crawler`):
    - Crawler indeksujący artykuły Wikipedii (Simple English)
    - Przygotowanie bazy danych SQLite zawierającej zaindeksowane artykuły

## 3. Realizacja głównych elementów zadania

### 3.1 Budowa słownika i macierzy term-document

Proces budowy słownika zaimplementowano poprzez funkcję `build_vocabulary()` w `src/preprocessing.py`:

- **Eliminacja stop-words**: Wykorzystano bibliotekę NLTK do usunięcia słów nieznaczących (np. "the", "and", "of").

- **Filtrowanie rzadkich terminów**: Zastosowano parametr `min_df` (domyślnie 5), eliminujący terminy występujące w mniej niż określonej liczbie dokumentów. Poprawia to jakość wyszukiwania, usuwając szum i pozostawiając jedynie terminy, które wprowadzają znaczenie semantyczne.

- **Filtrowanie powszechnych terminów**: Zastosowano parametr `max_df` (domyślnie 0.5), eliminujący terminy występujące w więcej niż określonym procencie dokumentów, co stanowi uzupełnienie do eliminacji stop-words.

Następnie zaimplementowano tworzenie macierzy term-document w funkcji `create_document_vectors()`:
- Wykorzystano format macierzy rzadkiej CSR z biblioteki SciPy

### 3.2 Transformacja IDF i normalizacja

W funkcji `apply_idf()` zaimplementowano transformację TF-IDF:
- Obliczanie wartości IDF zgodnie z formułą: `IDF(term) = log(N / df(term))`
- Zastosowanie tych wag do macierzy term-document

Dodatkowo zastosowano normalizację wektorów dokumentów, co jest kluczowe dla późniejszego obliczania podobieństwa cosinusowego.

### 3.3 Implementacja SVD

Zgodnie z wymaganiami, zaimplementowano redukcję wymiarowości przy użyciu SVD w pliku `src/svd.py`:
- Wykorzystanie funkcji `svds` z biblioteki SciPy
- Umożliwienie wyboru parametru k (wymiaru macierzy po redukcji)
- Zapisywanie wyników SVD dla różnych wartości k w celu przyspieszenia pracy

### 3.4 Funkcje wyszukiwania

Zaimplementowano dwie funkcje wyszukiwania:

1. **Wyszukiwanie tradycyjne** (`search_documents()`):
   - Konwersja zapytania na wektor w przestrzeni terminów
   - Obliczanie podobieństwa cosinusowego między zapytaniem a dokumentami
   - Sortowanie wyników według podobieństwa

2. **Wyszukiwanie z SVD** (`search_documents_svd()`):
   - Konwersja zapytania na wektor w przestrzeni terminów
   - Projekcja zapytania do przestrzeni pojęć (konceptów) poprzez macierz U
   - Obliczanie podobieństwa w zredukowanej przestrzeni
   - Sortowanie wyników według podobieństwa

## 4. Eksperymenty i analiza wyników

Przeprowadzono szereg eksperymentów porównujących:
- Wyszukiwanie z IDF i bez IDF
- Wyszukiwanie bez SVD oraz z SVD dla różnych wartości k (100, 200, 500, 1000, 2000)

### 4.1. Importy i konfiguracja

In [1]:
import sqlite3
import time

from IPython.display import Markdown, display
from sklearn.preprocessing import normalize
from src.preprocessing import apply_idf
from src.search import search_documents, search_documents_svd
from src.setup import (load_or_compute_svd, load_or_compute_term_doc_matrix,
                       load_or_compute_vocabulary)

DB_PATH = './data/simplified_wiki_index.db'

### 4.2. Budowanie słownika i macierzy term-document, IDF, normalizacja

In [2]:
vocabulary = load_or_compute_vocabulary()

term_doc_matrix, doc_ids = load_or_compute_term_doc_matrix(vocabulary)

print("\nApplying IDF transformation...")
start_time = time.time()
term_doc_matrix_idf, idf_values = apply_idf(term_doc_matrix)
print(f"TIME: {time.time() - start_time:.2f}s")

print("\nNormalizing matrices...")
start_time = time.time()
normalized_term_doc_matrix = normalize(term_doc_matrix, axis=1)
normalized_term_doc_matrix_idf = normalize(term_doc_matrix_idf, axis=1)
print(f"TIME: {time.time() - start_time:.2f}s")


Loading precomputed vocabulary from file...

Loading precomputed term-document matrix and document IDs from file...

Applying IDF transformation...
TIME: 0.19s

Normalizing matrices...
TIME: 0.13s


### 4.3. Porównanie wyników wyszukiwania dla różnych konfiguracji

In [3]:
test_queries = [
    "artificial intelligence",
    "quantum physics",
    "history of Poland",
    "climate change",
    "machine learning algorithms",
    "graph algorithms",
    "alan turing"
]

k_values = [100, 200, 500, 1000]
results_table = {}

for query in test_queries:
    results_table[query] = {}
    results = search_documents(query, normalized_term_doc_matrix_idf, vocabulary, doc_ids, idf_values)
    results_table[query]["no_svd_idf"] = results

for k_svd in k_values:
    svd_components = load_or_compute_svd(True, normalized_term_doc_matrix_idf, k_svd)
    for query in test_queries:
        results = search_documents_svd(query, svd_components, vocabulary, doc_ids, idf_values)
        results_table[query][f"svd_k={k_svd}_idf"] = results

for query in test_queries:
    results = search_documents(query, normalized_term_doc_matrix, vocabulary, doc_ids, None)
    results_table[query]["no_svd_no_idf"] = results

for k_svd in k_values:
    svd_components = load_or_compute_svd(False, normalized_term_doc_matrix, k_svd)
    for query in test_queries:
        results = search_documents_svd(query, svd_components, vocabulary, doc_ids, None)
        results_table[query][f"svd_k={k_svd}_no_idf"] = results

all_doc_ids = set()
for query in test_queries:
    for config_results in results_table[query].values():
        all_doc_ids.update([doc_id for doc_id, _ in config_results])

conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()
titles = {}
for doc_id in all_doc_ids:
    cursor.execute("SELECT title FROM pages WHERE id = ?", (doc_id,))
    title = cursor.fetchone()[0]
    titles[doc_id] = title
conn.close()

for query in test_queries:
    markdown_table = f"## Zapytanie: '{query}'\n\n"
    configs = ["bez SVD"] + [f"SVD k={k}" for k in k_values]
    markdown_table += f"|     | {' | '.join(configs)} |\n"
    markdown_table += f"| --- | {' | '.join(['---' for _ in configs])} |\n"

    row = ["**z IDF**"]

    for config in ["no_svd_idf"] + [f"svd_k={k}_idf" for k in k_values]:
        results = results_table[query][config] if results_table[query][config] else []
        cell_content = []

        for i, (doc_id, score) in enumerate(results):
            doc_title = titles[doc_id]
            if len(doc_title) > 30:
                doc_title = doc_title[:27] + "..."
            cell_content.append(f"{i+1}. ({score:.4f}) {doc_title}")

        row.append("<br>".join(cell_content) if cell_content else "---")

    markdown_table += f"| {' | '.join(row)} |\n"

    row = ["**bez IDF**"]

    for config in ["no_svd_no_idf"] + [f"svd_k={k}_no_idf" for k in k_values]:
        results = results_table[query][config] if results_table[query][config] else []
        cell_content = []

        for i, (doc_id, score) in enumerate(results):
            doc_title = titles[doc_id]
            if len(doc_title) > 30:
                doc_title = doc_title[:27] + "..."
            cell_content.append(f"{i+1}. ({score:.4f}) {doc_title}")

        row.append("<br>".join(cell_content) if cell_content else "---")

    markdown_table += f"| {' | '.join(row)} |\n"

    display(Markdown(markdown_table))


Loading precomputed SVD components (k=100, idf) from file...

Loading precomputed SVD components (k=200, idf) from file...

Loading precomputed SVD components (k=500, idf) from file...

Loading precomputed SVD components (k=1000, idf) from file...

Loading precomputed SVD components (k=100, no idf) from file...

Loading precomputed SVD components (k=200, no idf) from file...

Loading precomputed SVD components (k=500, no idf) from file...

Loading precomputed SVD components (k=1000, no idf) from file...


## Zapytanie: 'artificial intelligence'

|     | bez SVD | SVD k=100 | SVD k=200 | SVD k=500 | SVD k=1000 |
| --- | --- | --- | --- | --- | --- |
| **z IDF** | 1. (0.2354) List of artificial islands<br>2. (0.2311) Global Artificial Intellige...<br>3. (0.1934) Isabelle Ryl<br>4. (0.1859) Artificial language<br>5. (0.1637) Nick Bostrom<br>6. (0.1355) Roko's basilisk<br>7. (0.1312) Serviciul Român de Informaţii<br>8. (0.1228) Semantic Scholar<br>9. (0.1177) Palm Islands<br>10. (0.1146) Natural language generation | 1. (0.8319) MIT Lincoln Laboratory<br>2. (0.8286) Triton (malware)<br>3. (0.8260) Technologist<br>4. (0.8212) DARPA<br>5. (0.8157) National Institute of Stand...<br>6. (0.8107) Khanh D. Pham<br>7. (0.8091) Emerging technologies<br>8. (0.8085) Shakey the robot<br>9. (0.8073) Amy B. Smith<br>10. (0.8052) Laboratory | 1. (0.8088) DARPA<br>2. (0.7998) MIT Lincoln Laboratory<br>3. (0.7744) Technologist<br>4. (0.7523) Isabelle Ryl<br>5. (0.7461) ONERA<br>6. (0.7434) Heléne Huby<br>7. (0.7410) Laboratory<br>8. (0.7401) Defense Intelligence Agency<br>9. (0.7280) Dava Newman<br>10. (0.7237) Paola Santana | 1. (0.6888) Isabelle Ryl<br>2. (0.6824) Artificial language<br>3. (0.5976) DARPA<br>4. (0.5859) List of artificial islands<br>5. (0.5846) MIT Lincoln Laboratory<br>6. (0.5656) Natural language generation<br>7. (0.5575) Semantic Scholar<br>8. (0.5575) Artificial<br>9. (0.5517) SAS language<br>10. (0.5425) Nick Bostrom | 1. (0.7381) Isabelle Ryl<br>2. (0.7089) Artificial language<br>3. (0.6399) List of artificial islands<br>4. (0.6366) National Security Agency<br>5. (0.6139) United States Intelligence ...<br>6. (0.6139) SAS language<br>7. (0.6126) Counterintelligence<br>8. (0.6124) Defence Intelligence<br>9. (0.6101) Natural language generation<br>10. (0.6005) Intelligence officer |
| **bez IDF** | 1. (0.2296) Global Artificial Intellige...<br>2. (0.2241) List of artificial islands<br>3. (0.1930) Isabelle Ryl<br>4. (0.1770) Artificial language<br>5. (0.1627) Nick Bostrom<br>6. (0.1385) Serviciul Român de Informaţii<br>7. (0.1346) Roko's basilisk<br>8. (0.1220) Semantic Scholar<br>9. (0.1164) Director of the Central Int...<br>10. (0.1160) Natural language generation | 1. (0.8321) DARPA<br>2. (0.8300) MIT Lincoln Laboratory<br>3. (0.8290) Technologist<br>4. (0.8275) Triton (malware)<br>5. (0.8142) National Institute of Stand...<br>6. (0.8131) Khanh D. Pham<br>7. (0.8042) Shakey the robot<br>8. (0.8040) Emerging technologies<br>9. (0.8029) Amy B. Smith<br>10. (0.7990) Institute for the Study of War | 1. (0.8185) DARPA<br>2. (0.7960) MIT Lincoln Laboratory<br>3. (0.7775) Technologist<br>4. (0.7564) Defense Intelligence Agency<br>5. (0.7527) Isabelle Ryl<br>6. (0.7478) ONERA<br>7. (0.7431) Heléne Huby<br>8. (0.7339) Institute for the Study of War<br>9. (0.7323) Laboratory<br>10. (0.7281) Paola Santana | 1. (0.6919) Isabelle Ryl<br>2. (0.6668) Artificial language<br>3. (0.6122) DARPA<br>4. (0.5805) MIT Lincoln Laboratory<br>5. (0.5733) Natural language generation<br>6. (0.5648) List of artificial islands<br>7. (0.5605) Semantic Scholar<br>8. (0.5531) SAS language<br>9. (0.5453) Defense Intelligence Agency<br>10. (0.5432) Nick Bostrom | 1. (0.7389) Isabelle Ryl<br>2. (0.6940) Artificial language<br>3. (0.6560) National Security Agency<br>4. (0.6330) Defence Intelligence<br>5. (0.6322) Counterintelligence<br>6. (0.6318) United States Intelligence ...<br>7. (0.6210) List of artificial islands<br>8. (0.6197) Intelligence officer<br>9. (0.6192) Military Intelligence Direc...<br>10. (0.6157) SAS language |


## Zapytanie: 'quantum physics'

|     | bez SVD | SVD k=100 | SVD k=200 | SVD k=500 | SVD k=1000 |
| --- | --- | --- | --- | --- | --- |
| **z IDF** | 1. (0.3606) Quantum theory<br>2. (0.1803) Quantum nonlocality<br>3. (0.1715) Quantum fluctuation<br>4. (0.1637) Quantum<br>5. (0.1636) Quantum healing<br>6. (0.1603) Filament (astronomy)<br>7. (0.1603) Quantum dot<br>8. (0.1479) Stephen Wiesner<br>9. (0.1442) Quantum number<br>10. (0.1311) Cosmic Quantum Ray | 1. (0.9881) Physical<br>2. (0.9824) Quantum theory<br>3. (0.9814) Peter Higgs<br>4. (0.9743) Yoichiro Nambu<br>5. (0.9667) Paul Dirac<br>6. (0.9665) Quantum<br>7. (0.9657) James Cronin<br>8. (0.9633) Roberto Peccei<br>9. (0.9618) Grigory E. Volovik<br>10. (0.9616) Bryce DeWitt | 1. (0.9798) Physical<br>2. (0.9690) Roberto Peccei<br>3. (0.9613) Werner Heisenberg<br>4. (0.9604) Hans-Peter Dürr<br>5. (0.9592) Grigory E. Volovik<br>6. (0.9587) Quantum<br>7. (0.9573) Quantum theory<br>8. (0.9556) Eugene Wigner<br>9. (0.9552) Bryce DeWitt<br>10. (0.9539) James Cronin | 1. (0.9641) Physical<br>2. (0.9375) Quantum<br>3. (0.9256) Quantum theory<br>4. (0.9116) Roberto Peccei<br>5. (0.9050) E. C. George Sudarshan<br>6. (0.8987) Lev Pitaevskii<br>7. (0.8876) Entanglement<br>8. (0.8848) Werner Heisenberg<br>9. (0.8832) Physicist<br>10. (0.8741) Joseph Polchinski | 1. (0.9462) Quantum<br>2. (0.9432) Quantum theory<br>3. (0.9325) Physical<br>4. (0.8934) Werner Heisenberg<br>5. (0.8890) Classical physics<br>6. (0.8888) Born's rule<br>7. (0.8798) E. C. George Sudarshan<br>8. (0.8787) Lev Pitaevskii<br>9. (0.8678) Matrix mechanics<br>10. (0.8675) John Clauser |
| **bez IDF** | 1. (0.3045) Quantum theory<br>2. (0.1612) Physical<br>3. (0.1537) Quantum fluctuation<br>4. (0.1522) Quantum nonlocality<br>5. (0.1491) Quantum<br>6. (0.1408) Quantum healing<br>7. (0.1381) Applied physics<br>8. (0.1353) Quantum dot<br>9. (0.1353) Filament (astronomy)<br>10. (0.1249) Stephen Wiesner | 1. (0.9917) Physical<br>2. (0.9797) Peter Higgs<br>3. (0.9786) Quantum theory<br>4. (0.9716) Yoichiro Nambu<br>5. (0.9648) James Cronin<br>6. (0.9646) Grigory E. Volovik<br>7. (0.9627) Quantum<br>8. (0.9616) Roberto Peccei<br>9. (0.9608) Bryce DeWitt<br>10. (0.9607) Paul Dirac | 1. (0.9866) Physical<br>2. (0.9655) Roberto Peccei<br>3. (0.9627) Grigory E. Volovik<br>4. (0.9621) Werner Heisenberg<br>5. (0.9590) Hans-Peter Dürr<br>6. (0.9580) Eugene Wigner<br>7. (0.9571) Quantum<br>8. (0.9560) Bryce DeWitt<br>9. (0.9522) Quantum theory<br>10. (0.9495) Hugh David Politzer | 1. (0.9760) Physical<br>2. (0.9334) Quantum<br>3. (0.9136) Quantum theory<br>4. (0.9074) E. C. George Sudarshan<br>5. (0.9028) Roberto Peccei<br>6. (0.9014) Lev Pitaevskii<br>7. (0.8953) Physicist<br>8. (0.8831) Entanglement<br>9. (0.8812) Werner Heisenberg<br>10. (0.8753) Eugene Wigner | 1. (0.9527) Physical<br>2. (0.9377) Quantum<br>3. (0.9282) Quantum theory<br>4. (0.8859) E. C. George Sudarshan<br>5. (0.8834) Lev Pitaevskii<br>6. (0.8801) Classical physics<br>7. (0.8796) Physicist<br>8. (0.8762) John Clauser<br>9. (0.8750) Born's rule<br>10. (0.8749) Werner Heisenberg |


## Zapytanie: 'history of Poland'

|     | bez SVD | SVD k=100 | SVD k=200 | SVD k=500 | SVD k=1000 |
| --- | --- | --- | --- | --- | --- |
| **z IDF** | 1. (0.1176) Rzeszów–Jasionka Airport<br>2. (0.0882) Gorlice<br>3. (0.0882) Sulejówek<br>4. (0.0807) History of the world<br>5. (0.0807) World history<br>6. (0.0784) Tenczynek<br>7. (0.0784) Pleszew<br>8. (0.0706) San (river)<br>9. (0.0706) Pabianice<br>10. (0.0706) Kutno | 1. (0.8951) Sulejówek<br>2. (0.8906) Kazimierz Wielki University...<br>3. (0.8904) Prudnik<br>4. (0.8891) Chełm<br>5. (0.8868) Zielona Góra<br>6. (0.8833) Kraków<br>7. (0.8692) Silesian Voivodeship<br>8. (0.8690) Wadowice<br>9. (0.8669) Nowy Sącz<br>10. (0.8615) Verkhniy Studenyy | 1. (0.9727) Międzyrzec Podlaski<br>2. (0.9722) Lublin Voivodeship<br>3. (0.9698) Starogard Gdański<br>4. (0.9695) Tomaszów Lubelski<br>5. (0.9689) Masovian Voivodeship<br>6. (0.9687) Ostrów Mazowiecka<br>7. (0.9687) Ostrołęka<br>8. (0.9687) Stary Sącz<br>9. (0.9685) Białogard<br>10. (0.9683) Tenczynek | 1. (0.9558) Ostrołęka<br>2. (0.9476) Masovian Voivodeship<br>3. (0.9456) Bytom<br>4. (0.9422) Tomaszów Lubelski<br>5. (0.9419) Konin (disambiguation)<br>6. (0.9391) Lublin Voivodeship<br>7. (0.9384) Ostrów Mazowiecka<br>8. (0.9364) Tomaszów Mazowiecki<br>9. (0.9363) Piotrków Trybunalski<br>10. (0.9329) Radom | 1. (0.9175) Masovian Voivodeship<br>2. (0.9149) Ostrołęka<br>3. (0.9079) Ostrów Mazowiecka<br>4. (0.9078) Piotrków Trybunalski<br>5. (0.9078) Tomaszów Mazowiecki<br>6. (0.9026) Lublin Voivodeship<br>7. (0.8981) Tomaszów Lubelski<br>8. (0.8947) Stalowa Wola<br>9. (0.8942) Starachowice<br>10. (0.8934) Radlin |
| **bez IDF** | 1. (0.1019) Rzeszów–Jasionka Airport<br>2. (0.0988) History of the world<br>3. (0.0988) World history<br>4. (0.0764) Sulejówek<br>5. (0.0764) Gorlice<br>6. (0.0679) Pleszew<br>7. (0.0679) Tenczynek<br>8. (0.0659) Tokyo (disambiguation)<br>9. (0.0659) Architecture in Stockholm<br>10. (0.0611) Kutno | 1. (0.8673) Wadowice<br>2. (0.8639) Kraków<br>3. (0.8612) Jerzy Turonek<br>4. (0.8571) Kazimierz Wielki University...<br>5. (0.8460) Prudnik<br>6. (0.8455) Verkhniy Studenyy<br>7. (0.8448) Mińsk Mazowiecki<br>8. (0.8387) Katowice<br>9. (0.8381) Sulejówek<br>10. (0.8373) List of Jewish ghettos in G... | 1. (0.9567) Lublin Voivodeship<br>2. (0.9562) Międzyrzec Podlaski<br>3. (0.9545) Sokołów Podlaski<br>4. (0.9540) Starogard Gdański<br>5. (0.9539) Masovian Voivodeship<br>6. (0.9537) Tricity<br>7. (0.9529) Tomaszów Lubelski<br>8. (0.9523) Giżycko<br>9. (0.9522) Białogard<br>10. (0.9519) Ełk | 1. (0.9323) Ostrołęka<br>2. (0.9274) Masovian Voivodeship<br>3. (0.9229) Bytom<br>4. (0.9203) Konin (disambiguation)<br>5. (0.9194) Tomaszów Lubelski<br>6. (0.9173) Lublin Voivodeship<br>7. (0.9155) Ostrów Mazowiecka<br>8. (0.9134) Tomaszów Mazowiecki<br>9. (0.9127) Piotrków Trybunalski<br>10. (0.9104) Radom | 1. (0.8763) Masovian Voivodeship<br>2. (0.8692) Ostrołęka<br>3. (0.8647) Piotrków Trybunalski<br>4. (0.8645) Ostrów Mazowiecka<br>5. (0.8639) Tomaszów Mazowiecki<br>6. (0.8594) Lublin Voivodeship<br>7. (0.8537) Tomaszów Lubelski<br>8. (0.8519) Stalowa Wola<br>9. (0.8500) Starachowice<br>10. (0.8489) Radlin |


## Zapytanie: 'climate change'

|     | bez SVD | SVD k=100 | SVD k=200 | SVD k=500 | SVD k=1000 |
| --- | --- | --- | --- | --- | --- |
| **z IDF** | 1. (0.2760) Climate change denial<br>2. (0.1446) Glossary of climate change<br>3. (0.1222) Biarritz<br>4. (0.1149) Climate apocalypse<br>5. (0.1087) Climate commitment studies<br>6. (0.1048) Alpine climate<br>7. (0.1048) Climate of Nigeria<br>8. (0.1047) United Kingdom Climate Chan...<br>9. (0.1029) Polar climate<br>10. (0.0920) Department of Energy and Cl... | 1. (0.9516) Kottanad<br>2. (0.9439) Spit (landform)<br>3. (0.9413) Moder (river)<br>4. (0.9342) Lahai<br>5. (0.9287) Antony, Hauts-de-Seine<br>6. (0.9279) Kummannoor<br>7. (0.9268) Kuttoor<br>8. (0.9259) Koipuram<br>9. (0.9259) Cherickal<br>10. (0.9258) Kozhencherry East | 1. (0.9231) Climate change denial<br>2. (0.9005) Climate apocalypse<br>3. (0.8970) Glossary of climate change<br>4. (0.8793) Regan, North Dakota<br>5. (0.8711) Alpine climate<br>6. (0.8682) Climate of France<br>7. (0.8588) Oceanic climate<br>8. (0.8520) Climate of Nigeria<br>9. (0.8467) Guantánamo Province<br>10. (0.8397) Flagstaff, Arizona | 1. (0.9160) Climate change denial<br>2. (0.9072) Glossary of climate change<br>3. (0.8969) Oceanic climate<br>4. (0.8936) Climate apocalypse<br>5. (0.8867) Climate of France<br>6. (0.8848) Humid continental climate<br>7. (0.8826) Climate of Nigeria<br>8. (0.8802) Regan, North Dakota<br>9. (0.8777) Alpine climate<br>10. (0.8692) Flagstaff, Arizona | 1. (0.9204) Climate change denial<br>2. (0.8793) Glossary of climate change<br>3. (0.8787) Climate of Nigeria<br>4. (0.8649) Climate apocalypse<br>5. (0.8645) Santander, Spain<br>6. (0.8640) Humid continental climate<br>7. (0.8622) Biarritz<br>8. (0.8593) Tropical rainforest climate<br>9. (0.8533) Regan, North Dakota<br>10. (0.8513) Nancy, France |
| **bez IDF** | 1. (0.2675) Climate change denial<br>2. (0.1401) Glossary of climate change<br>3. (0.1074) Climate apocalypse<br>4. (0.1057) United Kingdom Climate Chan...<br>5. (0.1028) Biarritz<br>6. (0.0914) Climate commitment studies<br>7. (0.0892) Department of Energy and Cl...<br>8. (0.0881) Alpine climate<br>9. (0.0881) Climate of Nigeria<br>10. (0.0866) Polar climate | 1. (0.9346) Kottanad<br>2. (0.9313) Spit (landform)<br>3. (0.9275) Climate change denial<br>4. (0.9253) Glossary of climate change<br>5. (0.9252) Moder (river)<br>6. (0.9197) Climate apocalypse<br>7. (0.9153) Lahai<br>8. (0.9134) Unit<br>9. (0.9130) Rain shadow<br>10. (0.9122) Antony, Hauts-de-Seine | 1. (0.9263) Climate change denial<br>2. (0.9107) Climate apocalypse<br>3. (0.9072) Glossary of climate change<br>4. (0.8680) Alpine climate<br>5. (0.8678) Regan, North Dakota<br>6. (0.8626) Climate of France<br>7. (0.8446) Oceanic climate<br>8. (0.8358) Climate of Nigeria<br>9. (0.8357) United Kingdom Climate Chan...<br>10. (0.8284) Guantánamo Province | 1. (0.9146) Climate change denial<br>2. (0.9119) Glossary of climate change<br>3. (0.8966) Climate apocalypse<br>4. (0.8828) Oceanic climate<br>5. (0.8772) Climate of France<br>6. (0.8742) Humid continental climate<br>7. (0.8688) Alpine climate<br>8. (0.8681) Regan, North Dakota<br>9. (0.8636) Climate of Nigeria<br>10. (0.8555) Köppen climate classification | 1. (0.9171) Climate change denial<br>2. (0.8798) Glossary of climate change<br>3. (0.8641) Climate apocalypse<br>4. (0.8553) Climate of Nigeria<br>5. (0.8448) Humid continental climate<br>6. (0.8405) Santander, Spain<br>7. (0.8396) Tropical rainforest climate<br>8. (0.8380) Biarritz<br>9. (0.8356) Regan, North Dakota<br>10. (0.8295) Nancy, France |


## Zapytanie: 'machine learning algorithms'

|     | bez SVD | SVD k=100 | SVD k=200 | SVD k=500 | SVD k=1000 |
| --- | --- | --- | --- | --- | --- |
| **z IDF** | 1. (0.1854) Nearest neighbour algorithm<br>2. (0.1778) SHA hash functions<br>3. (0.1708) Adaptive replacement cache<br>4. (0.1210) Exponentiation by squaring<br>5. (0.1210) Holographic algorithm<br>6. (0.1210) Evolution strategy<br>7. (0.1177) Simplex algorithm<br>8. (0.1117) Las Vegas algorithm<br>9. (0.1089) Differential privacy<br>10. (0.1085) Algorithm | 1. (0.9378) Computable function<br>2. (0.9360) Number theory<br>3. (0.9337) Turing machine<br>4. (0.9337) Computer science<br>5. (0.9334) Non-deterministic turing ma...<br>6. (0.9311) Shor's algorithm<br>7. (0.9303) LZ77 and LZ78<br>8. (0.9289) One instruction set computer<br>9. (0.9265) Transformer (machine learni...<br>10. (0.9257) Cryptanalysis | 1. (0.9482) Number theory<br>2. (0.9385) One-way function<br>3. (0.9381) Communication Theory of Sec...<br>4. (0.9354) Shor's algorithm<br>5. (0.9340) Cryptanalysis<br>6. (0.9332) Kerckhoffs's principle<br>7. (0.9332) Cryptography<br>8. (0.9317) Cryptographic hash function<br>9. (0.9308) Diceware<br>10. (0.9274) Space-time tradeoff | 1. (0.8467) MD5<br>2. (0.8422) One-way function<br>3. (0.8411) Number theory<br>4. (0.8397) Random<br>5. (0.8299) Pseudorandom number generator<br>6. (0.8295) Key derivation function<br>7. (0.8281) Shor's algorithm<br>8. (0.8233) Password<br>9. (0.8230) Diffie-Hellman key exchange<br>10. (0.8224) SHA hash functions | 1. (0.7131) Algorithm<br>2. (0.7050) Colossus computer<br>3. (0.7001) Nearest neighbour algorithm<br>4. (0.6998) MD5<br>5. (0.6897) Random<br>6. (0.6838) SHA hash functions<br>7. (0.6814) Computer science<br>8. (0.6805) Pseudorandom number generator<br>9. (0.6791) Cryptography<br>10. (0.6789) Number theory |
| **bez IDF** | 1. (0.1459) Nearest neighbour algorithm<br>2. (0.1399) SHA hash functions<br>3. (0.1345) Adaptive replacement cache<br>4. (0.1050) Self-replicating machine<br>5. (0.0967) Learned, Mississippi<br>6. (0.0964) Unsupervised learning<br>7. (0.0952) Evolution strategy<br>8. (0.0952) Holographic algorithm<br>9. (0.0952) Exponentiation by squaring<br>10. (0.0945) JeOS | 1. (0.9256) Machine learning<br>2. (0.9253) Computer science<br>3. (0.9238) Non-deterministic turing ma...<br>4. (0.9204) Turing machine<br>5. (0.9183) Computability theory<br>6. (0.9129) One instruction set computer<br>7. (0.9100) Transformer (machine learni...<br>8. (0.9073) Information systems<br>9. (0.9071) CAPTCHA<br>10. (0.9071) Optical mark recognition | 1. (0.9003) Number theory<br>2. (0.8946) Kerckhoffs's principle<br>3. (0.8943) One-way function<br>4. (0.8942) Computer science<br>5. (0.8929) Colossus computer<br>6. (0.8916) Cryptanalysis<br>7. (0.8904) Communication Theory of Sec...<br>8. (0.8878) Cryptographic hash function<br>9. (0.8871) MD5<br>10. (0.8858) Password | 1. (0.7576) MD5<br>2. (0.7536) Random<br>3. (0.7533) One-way function<br>4. (0.7527) Colossus computer<br>5. (0.7468) Coding theory<br>6. (0.7397) Computer science<br>7. (0.7390) Key derivation function<br>8. (0.7366) Number theory<br>9. (0.7349) Space-time tradeoff<br>10. (0.7345) Pseudorandom number generator | 1. (0.6915) Non-deterministic turing ma...<br>2. (0.6641) Machine learning<br>3. (0.6593) Computer science<br>4. (0.6356) Algorithm<br>5. (0.6336) Colossus computer<br>6. (0.6276) Teaching machine<br>7. (0.6118) Nearest neighbour algorithm<br>8. (0.5975) Machine<br>9. (0.5917) Random<br>10. (0.5913) MD5 |


## Zapytanie: 'graph algorithms'

|     | bez SVD | SVD k=100 | SVD k=200 | SVD k=500 | SVD k=1000 |
| --- | --- | --- | --- | --- | --- |
| **z IDF** | 1. (0.2574) Planar graph<br>2. (0.2030) Nearest neighbour algorithm<br>3. (0.2023) Loop (graph theory)<br>4. (0.2023) Bimodal distribution<br>5. (0.2023) x-intercept<br>6. (0.1770) Eulerian path<br>7. (0.1659) SHA hash functions<br>8. (0.1593) Adaptive replacement cache<br>9. (0.1539) Graphing calculator<br>10. (0.1490) Grid paper | 1. (0.9786) Exponentiation by squaring<br>2. (0.9667) Truncated differential cryp...<br>3. (0.9658) Wavelet transform<br>4. (0.9640) XNOR gate<br>5. (0.9615) Modular exponentiation<br>6. (0.9603) Shor's algorithm<br>7. (0.9602) Matrix function<br>8. (0.9589) RSA algorithm<br>9. (0.9587) Multivalued function<br>10. (0.9574) Fixed-point theorem | 1. (0.9746) Exponentiation by squaring<br>2. (0.9741) Permutation<br>3. (0.9720) Modular exponentiation<br>4. (0.9669) Fermat's primality test<br>5. (0.9653) Chinese remainder theorem<br>6. (0.9648) Linear-feedback shift register<br>7. (0.9635) RSA algorithm<br>8. (0.9584) XOR gate<br>9. (0.9579) XNOR gate<br>10. (0.9571) One-way function | 1. (0.8869) Simplex algorithm<br>2. (0.8738) Permutation<br>3. (0.8504) Nearest neighbour algorithm<br>4. (0.8496) Number theory<br>5. (0.8457) Linear-feedback shift register<br>6. (0.8451) Shor's algorithm<br>7. (0.8432) Peter Montgomery (mathemati...<br>8. (0.8398) Minimum spanning tree<br>9. (0.8361) Hough transform<br>10. (0.8353) One-way function | 1. (0.8412) Simplex algorithm<br>2. (0.8161) Nearest neighbour algorithm<br>3. (0.7791) Algorithm<br>4. (0.7771) Minimum spanning tree<br>5. (0.7744) Hough transform<br>6. (0.7696) SHA hash functions<br>7. (0.7659) Holographic algorithm<br>8. (0.7642) Peter Montgomery (mathemati...<br>9. (0.7604) Number theory<br>10. (0.7562) Shor's algorithm |
| **bez IDF** | 1. (0.2496) Planar graph<br>2. (0.2079) Nearest neighbour algorithm<br>3. (0.1961) Bimodal distribution<br>4. (0.1961) x-intercept<br>5. (0.1961) Loop (graph theory)<br>6. (0.1716) Eulerian path<br>7. (0.1714) SHA hash functions<br>8. (0.1647) Adaptive replacement cache<br>9. (0.1492) Graphing calculator<br>10. (0.1445) Grid paper | 1. (0.9781) Exponentiation by squaring<br>2. (0.9677) Truncated differential cryp...<br>3. (0.9650) Wavelet transform<br>4. (0.9637) XNOR gate<br>5. (0.9616) Shor's algorithm<br>6. (0.9609) Modular exponentiation<br>7. (0.9602) RSA algorithm<br>8. (0.9593) Matrix function<br>9. (0.9581) Multivalued function<br>10. (0.9566) AND gate | 1. (0.9742) Permutation<br>2. (0.9731) Exponentiation by squaring<br>3. (0.9708) Modular exponentiation<br>4. (0.9663) Fermat's primality test<br>5. (0.9662) Linear-feedback shift register<br>6. (0.9657) RSA algorithm<br>7. (0.9643) Chinese remainder theorem<br>8. (0.9584) One-way function<br>9. (0.9584) XOR gate<br>10. (0.9575) Number theory | 1. (0.8871) Simplex algorithm<br>2. (0.8791) Permutation<br>3. (0.8596) Number theory<br>4. (0.8553) Linear-feedback shift register<br>5. (0.8551) Shor's algorithm<br>6. (0.8551) Nearest neighbour algorithm<br>7. (0.8524) Peter Montgomery (mathemati...<br>8. (0.8439) One-way function<br>9. (0.8431) RSA algorithm<br>10. (0.8399) Substitution-permutation ne... | 1. (0.8416) Simplex algorithm<br>2. (0.8206) Nearest neighbour algorithm<br>3. (0.7838) Algorithm<br>4. (0.7812) SHA hash functions<br>5. (0.7738) Hough transform<br>6. (0.7736) Peter Montgomery (mathemati...<br>7. (0.7721) Minimum spanning tree<br>8. (0.7708) Number theory<br>9. (0.7671) Holographic algorithm<br>10. (0.7660) Shor's algorithm |


## Zapytanie: 'alan turing'

|     | bez SVD | SVD k=100 | SVD k=200 | SVD k=500 | SVD k=1000 |
| --- | --- | --- | --- | --- | --- |
| **z IDF** | 1. (0.6081) Turing<br>2. (0.2146) Church–Turing thesis<br>3. (0.1721) Non-deterministic turing ma...<br>4. (0.1544) Alan Ball<br>5. (0.1428) Turing complete<br>6. (0.1243) CAPTCHA<br>7. (0.1240) Turing machine<br>8. (0.1181) Computability theory<br>9. (0.1140) XSLT<br>10. (0.1123) Alan Oshyer | 1. (0.8735) Jerome Hellman<br>2. (0.8728) Images (movie)<br>3. (0.8578) Space Master X-7<br>4. (0.8566) The Absent-Minded Professor<br>5. (0.8492) The Boys from Brazil (movie)<br>6. (0.8484) The Nutty Professor (1996 m...<br>7. (0.8479) Earthquake (movie)<br>8. (0.8441) Z.P.G.<br>9. (0.8423) Project X (movie)<br>10. (0.8400) Robert Bakewell | 1. (0.7806) Robert Bakewell<br>2. (0.7787) Richard Russell<br>3. (0.7781) Odd Man Out<br>4. (0.7739) John Backus<br>5. (0.7738) Space Master X-7<br>6. (0.7655) Robert Blake<br>7. (0.7551) Arabesque (1966 movie)<br>8. (0.7540) Richard Mason<br>9. (0.7527) Jerome Hellman<br>10. (0.7521) Derek O'Brien | 1. (0.7031) Richard Russell<br>2. (0.6780) Robert Blake<br>3. (0.6753) Turing<br>4. (0.6698) Richard Mason<br>5. (0.6653) James II<br>6. (0.6598) Robert Bakewell<br>7. (0.6441) John Smith<br>8. (0.6420) John Hughes<br>9. (0.6347) Young Winston<br>10. (0.6278) Derek O'Brien | 1. (0.6829) Turing<br>2. (0.5023) Joe Armstrong (programmer)<br>3. (0.4888) Mary Lee Woods<br>4. (0.4851) Computability theory<br>5. (0.4849) John Alan Robinson<br>6. (0.4841) Richard Russell<br>7. (0.4690) CAPTCHA<br>8. (0.4689) James H. Wilkinson<br>9. (0.4676) William Kahan<br>10. (0.4617) James II |
| **bez IDF** | 1. (0.5278) Turing<br>2. (0.1882) Alan Ball<br>3. (0.1863) Church–Turing thesis<br>4. (0.1494) Non-deterministic turing ma...<br>5. (0.1369) Alan Oshyer<br>6. (0.1369) Alan Taylor<br>7. (0.1255) Alan<br>8. (0.1239) Turing complete<br>9. (0.1189) Alan (surname)<br>10. (0.1123) CAPTCHA | 1. (0.9049) Images (movie)<br>2. (0.9018) Jerome Hellman<br>3. (0.8967) Space Master X-7<br>4. (0.8908) The Boys from Brazil (movie)<br>5. (0.8903) Earthquake (movie)<br>6. (0.8866) Project X (movie)<br>7. (0.8849) Richard Russell<br>8. (0.8829) Derek O'Brien<br>9. (0.8814) Odd Man Out<br>10. (0.8810) The Absent-Minded Professor | 1. (0.8309) Richard Russell<br>2. (0.8245) Robert Bakewell<br>3. (0.8238) Odd Man Out<br>4. (0.8230) Robert Blake<br>5. (0.8207) Space Master X-7<br>6. (0.8127) Derek O'Brien<br>7. (0.8086) Britannia Hospital<br>8. (0.8046) Richard Mason<br>9. (0.8001) The Keys of the Kingdom (mo...<br>10. (0.7961) School for Secrets | 1. (0.7558) Richard Russell<br>2. (0.7276) Robert Blake<br>3. (0.7226) Richard Mason<br>4. (0.7077) James II<br>5. (0.6983) Robert Bakewell<br>6. (0.6908) John Hughes<br>7. (0.6891) Young Winston<br>8. (0.6865) John Smith<br>9. (0.6852) Derek O'Brien<br>10. (0.6830) Brothers in Law (movie) | 1. (0.5787) Turing<br>2. (0.5297) Richard Russell<br>3. (0.4952) John Hughes<br>4. (0.4949) James II<br>5. (0.4887) Robert Blake<br>6. (0.4683) John Smith<br>7. (0.4673) Joe Armstrong (programmer)<br>8. (0.4641) Alan Ball<br>9. (0.4627) The Elephant Man (movie)<br>10. (0.4626) John Parratt |


### 4.4. Najważniejsze wnioski z eksperymentów

#### **Wpływ IDF**
W przeprowadzonych eksperymentach zaobserwowano, że wpływ zastosowania IDF jest bardziej subtelny niż można by oczekiwać. Możliwe przyczyny:
- **Wstępne filtrowanie słownika** - Poprzez eliminację stop-words oraz filtrowanie terminów przy tworzeniu słownika, terminy o potencjalnie najniższych wagach IDF mogły zostać usunięte jeszcze zanim IDF zostało zastosowane.
- **Charakter źródła dokumentów** - Artykuły z Simple English Wikipedia charakteryzują się ograniczonym i bardziej jednolitym słownictwem, co może wpływać na mniejsze zróżnicowanie wag IDF.

#### **Wpływ SVD**
- **Semantyczne powiązania** - Wyniki z SVD często zawierają dokumenty, które niekoniecznie zawierają dokładne terminy z zapytania, ale są semantycznie powiązane.
- **Zwiększenie poziomów dopasowania** - Zastosowanie SVD znacząco zwiększa poziom dopasowania wyników wyszukiwania - podobieństwo jest mierzone w przestrzeni konceptów, która jest bardziej zwarta od przestrzeni terminów.
- **Redukcja szumu** - W niektórych przypadkach SVD eliminuje z topowych wyników dokumenty, które mają przypadkowe dopasowania terminów, ale nie są semantycznie związane z tematem zapytania.

#### **Wpływ wartości k w SVD**
- **Niższe wartości k (100-200)** - Większa generalizacja, pojawiają się wyniki bardziej odległe semantycznie, często niezwiązane z wyszukiwaną frazą
- **Wyższe wartości k (500-1000)** - Z reguły lepsze (subiektywnie) dopasowanie wyników

## 5. Wnioski ogólne

Projekt demonstruje kompromisy między tradycyjnym podejściem TF-IDF a metodą Latent Semantic Indexing. Oba podejścia mają swoje zalety i wady:
- **Tradycyjne wyszukiwanie**: Szybsza inicjalizacja, dokładniejsze dopasowanie dla konkretnych terminów, ale ograniczona zdolność do znajdowania semantycznie powiązanych dokumentów.
- **Wyszukiwanie z SVD**: Potencjał do znajdowania semantycznie powiązanych dokumentów nawet przy braku dokładnego dopasowania terminów, kosztem większej złożoności obliczeniowej i potrzeby starannego doboru parametrów.

Projekt skutecznie łączy zaawansowane techniki algebry liniowej z praktycznym rozwiązaniem problemu wyszukiwania informacji, dostarczając funkcjonalny i wydajny system.